<h1>Welcome to the Resource Watch Duplicate WMS Layers Tool!</h1>

In [ ]:
#Import necessary layers
#!pip install LMIPy
import LMIPy as lmi
import os
import json
import geopandas as gpd
from IPython.display import display
from pprint import pprint
from datetime import datetime
import numpy as np
import json
import pandas as pd
import copy
import dotenv
#insert the location of your .env file here:
dotenv.load_dotenv() 

<font color=red>**ACTION REQUIRED** <br>
<font color=lightblue>**Enter your API Token Below**</font>

In [ ]:
#API token needed to make changes
API_TOKEN = os.getenv('RW_API_KEY')

<font color=red>**ACTION REQUIRED** <br>
<font color=lightblue>**Enter your dataset ID below** <br>
The dataset will be printed out below, then the first layer in the list of layers, followed by the number of layers in the dataset. We will duplicate the first layer.</font>

In [ ]:
#Select Dataset and Layer to Clone, here we just chose the first layer
dataset_id = '' #NEEDS TO BE THE LONG ONE LIKE 6cfd6255-609e-4922-8709-5aaad0db9fae
dataset = lmi.Dataset(dataset_id) #select the dataset by changing the ID
long_dataset_id = dataset.id

print('Dataset selected is below: ')
display(dataset)
# print(dataset.layers)
layer_to_clone = dataset.layers[0]   #Chose the first layer

print('First layer is below:')
display(layer_to_clone)

print("Number of layers in dataset: {}".format(len(dataset.layers)))


__________________________________________________________________________________________________________________
<font color=lightblue>**Now we will get the attributes that need to change**<br>
This includes the layer name, description, layerConfig, order, timeline label, url, and tiles <br>
We'll first print them out so you can see</font>

In [ ]:
#Gathering the right attributes that need to change
name = layer_to_clone.attributes['name']
description = layer_to_clone.attributes['description']
layerConfig = layer_to_clone.attributes['layerConfig']
order = str(layerConfig['order'])
timeLineLabel = layerConfig['timelineLabel']
url = layerConfig['url']
tiles = layerConfig['source']['tiles']

#Print them out
print('Name: \n{} \n'.format(name))
print('Description: \n{} \n'.format(description))
print('URL: \n{} \n'.format(url))
print('Tile: \n{} \n'.format(tiles))
print('Order Value: \n{} \n'.format(order))
print('Timeline Label: \n{} \n'.format(timeLineLabel))


<h1>Here we will replace the values in the attributes and decide what to loop over.</h1>

<font color=red>**ACTION REQUIRED**</font> <br>
<font color=lightblue>**Enter the text you would like to replace in the attributes above in the variable 'replace_string'** <br>
**AND enter the list of values you'd like to replace the search string with in the variable 'loop_values'.**<br>
The next cell will search for that string and replace it with a {} so it can be filled in later.<br>
Then we will loop over the values in 'loop_values' and replace the {} in the attributes with the values.</font>

In [ ]:
loop_values = pd.read_csv('loop_values.csv')
# Replace this value with the string you'd like to replace in the attributes
# replace_string = '45' #e.g., 1990
replace_strings = loop_values.columns.to_list()

# Enter the list you want to loop over to create new layers
# loop_values = np.arange(1951,2021)

#Print out the values so you can double check them
print('Replace String:\n{}'.format(replace_strings))
print('Loop Values:\n{}'.format(loop_values.to_string(header=False)))

<font color=lightblue>**Above are the values you entered. Double check that they look okay.**</font>

<font color=lightblue>**Next we will print out the attributes after replacing the 'replace_string' with '{}'**</font>

In [ ]:
name_convention = name
description_convention = description
order_convention = order
timeLineLabel_convention = timeLineLabel
tile_convention = tiles
url_convention = url 

for replace_string in replace_strings:
    name_convention = name_convention.replace(replace_string,'{}')
    description_convention = description_convention.replace(replace_string,'{}')
    order_convention = order_convention.replace(replace_string,'{}')
    url_convention = url_convention.replace(replace_string,'{}')
    tile_convention = str(tile_convention).replace(replace_string,'{}')
    timeLineLabel_convention = timeLineLabel_convention.replace(replace_string,'{}')

print('Name: \n{} \n'.format(name_convention))
print('Description: \n{} \n'.format(description_convention))
print('URL: \n{} \n'.format(url_convention))
print('Tiles: \n{} \n'.format(tile_convention))
print('Order Value: \n{} \n'.format(order_convention))
print('Timeline Label: \n{} \n'.format(timeLineLabel_convention))


<font color=lightblue>**Check out the values above, do they look okay? The places with a '{}' will be replaced when we loop over the values in 'loop_values'**</font>

In [ ]:
#If any of the variables above are not okay, replace it in this cell

<h1>Now we will loop through and create new layers!</h1>

<font color=lightblue>**The cell below loops through the values in 'loop_values' and replaces all of the {} with the value in each of the attributes. It will skip over a layer if a layer with that name already exists. When running the cell bellow, it sometimes returns an error that it times out, just run the cell again.**</font>

In [ ]:
# SOMETIMES WE GET AN ERROR THAT IT TIMES OUT, JUST RUN THE CELL AGAIN

# Load in dataset again to have the most updated version
dataset = lmi.Dataset(dataset_id)

# Get the names of all the layers
layers = dataset.layers
layer_names = list(map(lambda x: x.attributes['name'], layers))

# Loop over loop values
for index, new_values in loop_values.iterrows():
    new_layer_name = name
    new_description = description
    new_url = url
    new_tiles = tiles
    new_order = order
    new_timelineLabel = timeLineLabel

    for replace_string in replace_strings:

        # Get layer name
        new_layer_name = new_layer_name.replace(replace_string, str(new_values[replace_string]))  
        
        new_description = new_description.replace(replace_string, str(new_values[replace_string]))
        new_url = new_url.replace(replace_string, str(new_values[replace_string]))
        new_tiles = str(new_tiles).replace(replace_string, str(new_values[replace_string]))
        new_timelineLabel = new_timelineLabel.replace(replace_string, str(new_values[replace_string]))
        new_order = int(str(new_order).replace(replace_string, str(new_values[replace_string])))
        
    # Check that layer does not already exist
    if new_layer_name not in layer_names:    
        # Print those bad boys out
        print('NEW LAYER NAME: "{}"'.format(new_layer_name))
        print('New DESCRIPTION: "{}"'.format(new_description))
        print('NEW URL: "{}"'.format(new_url))
        print('NEW TILES: "{}"'.format(new_tiles))
        print('NEW TIMELINE LABEL: "{}"'.format(new_timelineLabel))
        print('NEW ORDER VALUE: "{}"'.format(new_order))
        
        #Clone layer
        clone_attributes = {
            'name': new_layer_name,
            'description': new_description
            }
        new_layer = layer_to_clone.clone(token=API_TOKEN, env='production', layer_params=clone_attributes, target_dataset_id=dataset_id)
        #Replace layer attributes with new values
        layerConfig = new_layer.attributes['layerConfig']
        layerConfig['url'] = new_url
        layerConfig['source']['tiles'] = new_tiles
        layerConfig['order'] = new_order
        layerConfig['timelineLabel'] = new_timelineLabel
        payload = {
        'layerConfig': {
            **layerConfig
            }
        }
        new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
        
        new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
            
        # Replace layer name and description
        payload = {
            'name': new_layer_name,
            'description': new_description
        }
        new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
        print(new_layer)
        print('\n')
            
    else:
        print('Layer "{}" already exists!'.format(new_layer_name))
        

In [ ]:
# Use this section to delete layers if they are stuck in staging or you want to bulk delete layers

# # UNCOMMENT THIS IF YOU WANT TO DELETE A SPECIFIC LAYER
# # Select layer by layer id:
# layer = lmi.Layer('c3075c5a-5567-4b09-bc0d-96ed1673f8b6')
# layer.delete(token=API_TOKEN)

# # UNCOMMENT THIS SECTION IF YOU WANT TO DELETE ALL LAYERS EXCEPT THE FIRST ONE
# # Enter dataset id (the long one)
# dataset_id = 'd17e6978-0848-4a13-ba05-6f4af04ac7d1' #THIS NEEDS TO BE THE LONG STRING ONE
# dataset = lmi.Dataset(dataset_id) 
# for i, layer in enumerate(dataset.layers):
#     if i<len(dataset.layers)-1:
#         display(layer)
#         layer.delete(token=API_TOKEN)